In [2]:
# ======================================================================
# CÉLULA ÚNICA, COMPLETA E CORRIGIDA
# ======================================================================

# --- Bloco 0: Importações e Limpeza de Sessão ---
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, backend as K
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import json
import numpy as np
import os

# BOA PRÁTICA: Limpa a sessão do Keras para evitar conflitos de nomes de execuções anteriores.
K.clear_session()

# --- Bloco 1: Definição das Camadas Customizadas "Anônimas" ---

# A classe 'No', otimizada para não gerenciar nomes internamente.
class No(layers.Layer):
    def __init__(self, layers_config, **kwargs):
        super(No, self).__init__(**kwargs)
        self.layers_config = layers_config
        self.internal_layers = []
        self.add_layer = layers.Add()
        for i, layer_info in enumerate(self.layers_config):
            # Deixamos o Keras nomear as camadas internas para garantir nomes únicos e válidos.
            self.internal_layers.append(
                getattr(layers, layer_info['type'])(**layer_info['config'])
            )

    def call(self, inputs):
        tensor_atual = inputs
        # Se a entrada for uma lista de tensores (ex: vinda da camada Add),
        # a camada 'add_layer' os soma antes de prosseguir.
        if isinstance(inputs, list) and len(inputs) > 1:
            tensor_atual = self.add_layer(inputs)

        # Aplica as camadas internas em sequência
        for layer in self.internal_layers:
            tensor_atual = layer(tensor_atual)
        return tensor_atual

    def get_config(self):
        config = super(No, self).get_config()
        config.update({"layers_config": self.layers_config})
        return config

# A classe 'Caminho', sem alterações.
class Caminho(layers.Layer):
    def __init__(self, **kwargs):
        super(Caminho, self).__init__(**kwargs)

    def build(self, input_shape):
        feature_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(feature_dim,), initializer="glorot_uniform", trainable=True, name="kernel")
        self.bias = self.add_weight(shape=(feature_dim,), initializer="zeros", trainable=True, name="bias")

    def call(self, inputs):
        return (inputs * self.kernel) + self.bias

    def get_config(self):
        config = super(Caminho, self).get_config()
        return config

# --- Bloco 2: Preparação dos Dados ---

# Criando um dataset de exemplo para que o código seja executável
dummy_dataset = [
    {"texto": "este produto é incrível e funciona bem", "intencao": "positivo"},
    {"texto": "não gostei, quebrou no primeiro dia", "intencao": "negativo"},
    {"texto": "atendimento ao cliente foi horrível", "intencao": "negativo"},
    {"texto": "recomendo fortemente a todos", "intencao": "positivo"},
    {"texto": "poderia ser melhor, mas é aceitável", "intencao": "neutro"},
    {"texto": "o preço é justo pela qualidade", "intencao": "neutro"},
    {"texto": "superou minhas expectativas", "intencao": "positivo"},
    {"texto": "decepcionado com a compra", "intencao": "negativo"},
    {"texto": "é um item ok, nem bom nem ruim", "intencao": "neutro"},
    {"texto": "a entrega foi muito rápida", "intencao": "positivo"}
] * 10 # Multiplicar para ter um volume mínimo para treino

# Salva o dataset em um arquivo JSON
with open('dataset_para_treino.json', 'w', encoding='utf-8') as f:
    json.dump(dummy_dataset, f, ensure_ascii=False, indent=4)

# Carrega os dados
with open('dataset_para_treino.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# Extrai textos e intenções
textos = [item['texto'] for item in dataset]
intencoes = [item['intencao'] for item in dataset]

# Mapeia intenções para índices e vice-versa
mapa_intencoes = {intencao: i for i, intencao in enumerate(set(intencoes))}
mapa_indices = {i: intencao for intencao, i in mapa_intencoes.items()}
num_classes = len(mapa_intencoes)

# Tokeniza os textos
MAX_WORDS = 1000
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(textos)
sequencias = tokenizer.texts_to_sequences(textos)
vocab_size = len(tokenizer.word_index) + 1

# Padroniza o comprimento das sequências
MAX_LEN = 20
entradas = pad_sequences(sequencias, maxlen=MAX_LEN, padding='post', truncating='post')

# Converte as saídas para o formato categórico (one-hot encoding)
saidas_indices = np.array([mapa_intencoes[i] for i in intencoes])
saidas = to_categorical(saidas_indices, num_classes=num_classes)

# Divide em treino e teste
tamanho_teste = int(len(entradas) * 0.2)
xs_treino, xs_teste = entradas[:-tamanho_teste], entradas[-tamanho_teste:]
ys_treino, ys_teste = saidas[:-tamanho_teste], saidas[-tamanho_teste:]

print(f"Dados prontos: {len(xs_treino)} amostras de treino, {len(xs_teste)} de teste.")
print(f"Vocabulário: {vocab_size} palavras. Classes: {num_classes}.")


# --- Bloco 3: Definição das Arquiteturas "Anônimas" ---

def construir_myceliumai_anonima(vocab_size, embedding_dim, max_len, num_classes):
    input_layer = layers.Input(shape=(max_len,))

    # Todas as camadas agora são anônimas (sem o parâmetro 'name')
    embedding_layer = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)
    lstm1_layer = layers.LSTM(32, return_sequences=True)
    lstm2_layer = layers.LSTM(32)
    no_entrada_layer = No(layers_config=[{'type': 'Dense', 'config': {'units': 128, 'activation': 'relu'}}, {'type': 'Dropout', 'config': {'rate': 0.4}}])
    caminho_a_layer = Caminho()
    caminho_b_layer = Caminho()
    no_oculto_a_layer = No(layers_config=[{'type': 'Dense', 'config': {'units': 64, 'activation': 'relu'}}])
    no_oculto_b_layer = No(layers_config=[{'type': 'Dense', 'config': {'units': 64, 'activation': 'relu'}}])
    add_layer = layers.Add()
    no_agregador_layer = No(layers_config=[{'type': 'Dense', 'config': {'units': 32, 'activation': 'relu'}}])
    no_saida_layer = No(layers_config=[{'type': 'Dense', 'config': {'units': num_classes, 'activation': 'softmax'}}])

    x = embedding_layer(input_layer)
    x = lstm1_layer(x)
    x = lstm2_layer(x)
    saida_no_entrada = no_entrada_layer(x)
    ramo_a = caminho_a_layer(saida_no_entrada)
    ramo_b = caminho_b_layer(saida_no_entrada)
    saida_oculto_a = no_oculto_a_layer(ramo_a)
    saida_oculto_b = no_oculto_b_layer(ramo_b)
    agregado = add_layer([saida_oculto_a, saida_oculto_b])
    saida_agregador = no_agregador_layer(agregado)
    output_final = no_saida_layer(saida_agregador)

    # O modelo também é criado sem um nome explícito
    model = models.Model(inputs=input_layer, outputs=output_final)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def construir_modelo_padrao(vocab_size, embedding_dim, max_len, num_classes):
    model = models.Sequential([
        # Camadas também anônimas
        layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
        layers.LSTM(32, return_sequences=True),
        layers.LSTM(32),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# ======================================================================
# Bloco 4: O Duelo Final (Usando as funções anônimas)
# ======================================================================
model_micelial = construir_myceliumai_anonima(vocab_size, 64, MAX_LEN, num_classes)
model_padrao = construir_modelo_padrao(vocab_size, 64, MAX_LEN, num_classes)

print("### MyceliumAI (Anônima) ###")
model_micelial.summary()
print("\n### Modelo Padrão ###")
model_padrao.summary()

early_stopping = callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)

print("\n--- Treinando MyceliumAI (Anônima) ---")
history_micelial = model_micelial.fit(
    xs_treino, ys_treino,
    epochs=50,
    batch_size=32,
    validation_data=(xs_teste, ys_teste),
    callbacks=[early_stopping],
    verbose=1
)

print("\n--- Treinando Modelo Padrão ---")
history_padrao = model_padrao.fit(
    xs_treino, ys_treino,
    epochs=50,
    batch_size=32,
    validation_data=(xs_teste, ys_teste),
    callbacks=[early_stopping],
    verbose=1
)

# Pega a melhor acurácia de validação que cada modelo atingiu durante o treino
acc_micelial = np.max(history_micelial.history['val_accuracy']) if history_micelial.history['val_accuracy'] else 0
acc_padrao = np.max(history_padrao.history['val_accuracy']) if history_padrao.history['val_accuracy'] else 0

print("\n========================================")
print("          PLACAR FINAL DA BATALHA")
print("========================================")
print(f"  MELHOR ACURÁCIA MYCELIUMAI: {acc_micelial*100:.2f}%")
print(f"  MELHOR ACURÁCIA REDE PADRÃO: {acc_padrao*100:.2f}%")
print("========================================")

if acc_micelial > acc_padrao:
    print("\nA Mycelium AI se mostrou mais eficaz neste caso")
elif acc_padrao > acc_micelial:
    print("\n🏁 A Rede Padrão se mostrou mais eficaz neste caso.")
else:
    print("\n🤝 Empate técnico! Um resultado impressionante.")

# ======================================================================
# BLOCO 5: SALVANDO O MODELO CAMPEÃO
# ======================================================================

# Vamos salvar o modelo que teve a melhor performance, seja ele qual for.
print("\n--- Salvando o modelo com a melhor performance ---")

if acc_micelial >= acc_padrao:
    print("O campeão é a MyceliumAI! Salvando o modelo...")
    modelo_final = model_micelial
else:
    print("O campeão é a Rede Padrão! Salvando o modelo...")
    modelo_final = model_padrao

# Salva o modelo no formato Keras nativo, que é o mais robusto.
modelo_final.save("modelo_sentimentos_campeao.keras")

# Também precisamos salvar o Tokenizer, que é o nosso "dicionário".
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("\n✅ Modelo ('modelo_sentimentos_campeao.keras') e Tokenizer ('tokenizer.pickle') salvos com sucesso!")

Dados prontos: 80 amostras de treino, 20 de teste.
Vocabulário: 49 palavras. Classes: 3.
### MyceliumAI (Anônima) ###


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 20, 64)    │      3,136 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 20, 32)    │     12,416 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 32)        │      8,320 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ no (No)             │ (None, 128)       │      4,224 │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ caminho (Caminho)   │ (None, 128)       │        256 │ no[0][0]          │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ caminho_1 (Caminho) │ (None, 128)       │        256 │ no[0][0]          │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ no_1 (No)           │ (None, 64)        │      8,256 │ caminho[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ no_2 (No)           │ (None, 64)        │      8,256 │ caminho_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 64)        │          0 │ no_1[0][0],       │
│                     │                   │            │ no_2[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ no_3 (No)           │ (None, 32)        │      2,080 │ add_3[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ no_4 (No)           │ (None, 3)         │         99 │ no_3[0][0]        │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 47,299 (184.76 KB)

 Trainable params: 47,299 (184.76 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


--- Treinando MyceliumAI (Anônima) ---
Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 362ms/step - accuracy: 0.3883 - loss: 1.0987 - val_accuracy: 0.4000 - val_loss: 1.0963
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.3641 - loss: 1.0971 - val_accuracy: 0.4000 - val_loss: 1.0949
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.3797 - loss: 1.0955 - val_accuracy: 0.4000 - val_loss: 1.0931
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.3406 - loss: 1.0993 - val_accuracy: 0.4000 - val_loss: 1.0917
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4109 - loss: 1.0901 - val_accuracy: 0.4000 - val_loss: 1.0896
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.3875 - loss: 1.0922 - val_accuracy: 0.4000 - val_loss: 1.0879
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.

--- Treinando Modelo Padrão ---
Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 332ms/step - accuracy: 0.3672 - loss: 1.0982 - val_acc